In [136]:
#!pip3 install eutils
#!pip3 install pycountry-convert

In [189]:
import pandas as pd
from eutils import Client
from datetime import datetime
import os
import pycountry_convert as pc
import re

In [96]:
with open('seq_accession_numbers.txt') as f:
    acc_nums = [line.rstrip() for line in f]

In [98]:
len(acc_nums)

99

In [90]:
info_dict = {}

In [87]:
gene_id = 'NC_003045.1'

In [91]:
info_dict[gene_id] = {}

In [88]:
ec = Client(api_key=os.environ.get("NCBI_API_KEY", None))

In [89]:
egs = ec.efetch(db='nuccore', id=gene_id)

In [64]:
dir(egs)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_root_tag',
 '_xml_root',
 'gbseqs']

In [65]:
entry = egs.gbseqs[0]#.acv

In [66]:
dir(entry)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_root_tag',
 '_xml_root',
 'acv',
 'cds',
 'comment',
 'created',
 'definition',
 'exons',
 'features',
 'gene',
 'genes',
 'gi',
 'length',
 'locus',
 'moltype',
 'organism',
 'other_seqids',
 'sequence',
 'updated']

In [67]:
for attr in dir(entry):
    if not attr.startswith('__'):
        print(attr)

_root_tag
_xml_root
acv
cds
comment
created
definition
exons
features
gene
genes
gi
length
locus
moltype
organism
other_seqids
sequence
updated


Save info into info dict

In [139]:
def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

In [141]:
country_to_continent('Venezuela')

'South America'

In [344]:
def fill_info_dict(acc_nums:list):
    info_dict = {}
    for counter, acc_num in enumerate(acc_nums):  #iterate over accession numbers in acc_nums, retrieve info from db and fill info_dict
        gene_id = acc_num
        info_dict[gene_id] = {}
        
        ec = Client(api_key=os.environ.get("NCBI_API_KEY", None))
        egs = ec.efetch(db='nuccore', id=gene_id)
        entry = egs.gbseqs[0]
        try:
            entry.acv
            info_dict[gene_id]['acv']=entry.acv
        except:
            info_dict[gene_id]['acv']='No information'

        try:
            entry.comment
            complete_comment=entry.comment
            regex = '##Assembly-Data-START## ; (.*) ; ##Assembly-Data-END##' 
            result = re.search(regex, complete_comment)
            comment = result.group(1)
            info_dict[gene_id]['comment']=comment
        except:
            info_dict[gene_id]['comment']=None

        try:
            entry.created
            info_dict[gene_id]['created']=entry.created
        except:
            info_dict[gene_id]['created']='No information'

        try:
            #save definition
            entry.definition
            info_dict[gene_id]['definition']=entry.definition
            
            #extract strain from definition using regex
            complete_string = entry.definition
            #note: 'respiratory' and ' isolate' are non-capturing groups, so that the strain-string will be the only (and first) capturing group
            result = re.search('Bovine(?: respiratory)? coronavirus (?:isolate )?(.*), complete genome', complete_string)
            strain = result.group(1)
            if strain.replace(' ','').isalpha():  #isalpha() does not consider \s --> remove whitespace before
                strain = s
            else:
                pass
            info_dict[gene_id]['strain'] = strain
        except:
            info_dict[gene_id]['definition']='No information'
            info_dict[gene_id]['strain']='No information'

        try:
#            if 'country' not in entry.features.source.qualifiers.keys():
#                print(f'No country passed for acc_num: {gene_id}')
            for k, v in entry.features.source.qualifiers.items():
                if k == 'country':
                    if len(re.split(':\s*', v)) > 1:
                        country, division = re.split(':\s*', v)
                        info_dict[gene_id][k] = country
                        info_dict[gene_id]['division'] = division
                    else:
                        info_dict[gene_id][k] = v
                else:
                    info_dict[gene_id][k] = v
        except:
            info_dict[gene_id][k]='No additional information'

        try:
            entry.gene
            info_dict[gene_id]['gene']=entry.gene
        except:
            info_dict[gene_id]['gene']='No information'

        try:
            entry.gi
            info_dict[gene_id]['gi']=entry.gi
        except:
            info_dict[gene_id]['gi']='No information'

        try:
            entry.length
            info_dict[gene_id]['length']=entry.length
        except:
            info_dict[gene_id]['length']='No information'

        try:
            entry.locus
            info_dict[gene_id]['locus']=entry.locus
        except:
            info_dict[gene_id]['locus']='No information'

        try:
            entry.moltype
            info_dict[gene_id]['moltype']=entry.moltype
        except:
            info_dict[gene_id]['moltype']='No information'

        try:
            entry.organism
            info_dict[gene_id]['organism']=entry.organism
        except:
            info_dict[gene_id]['organism']='No information'

        try:
            entry.other_seqids
            info_dict[gene_id]['other_seqids']=entry.other_seqids
        except:  
            info_dict[gene_id]['other_seqids']='None found'

        try:
            entry.sequence
            info_dict[gene_id]['sequence']=entry.sequence
        except:  
            info_dict[gene_id]['sequence']='No information'

        try:
            entry.updated
            info_dict[gene_id]['updated']=entry.updated
        except:  
            info_dict[gene_id]['updated']='No information'

        if 'updated' in info_dict[gene_id].keys():
            date_unformatted = info_dict[gene_id]['updated']
            info_dict[gene_id]['date']=datetime.strptime(date_unformatted, '%d-%b-%Y').strftime('%Y-%m-%d')
        elif 'created' in info_dict[gene_id].keys():
            date_unformatted = info_dict[gene_id]['created']
            info_dict[gene_id]['date']=datetime.strptime(date_unformatted, '%d-%b-%Y').strftime('%Y-%m-%d')
        else:
            info_dict[gene_id]['date']='20XX-XX-XX'  #TO DO: change this to correct form (sth like 2020.XX.XX)
            
        if 'country' in info_dict[gene_id].keys() and type(info_dict[gene_id]['country'])!=float and info_dict[gene_id]['country']!='No additional information':
            country = info_dict[gene_id]['country']
            print(country)
            info_dict[gene_id]['region'] = country_to_continent(country)
        else:
            info_dict[gene_id]['region'] = "No information"

            
    return info_dict

In [151]:
acc_nums[:5]

['MW711287.1', 'MN982199.1', 'MN982198.1', 'NC_003045.1', 'LC494178.1']

In [345]:
info_dict = fill_info_dict(acc_nums)

China


China


China


Japan


Japan


...

USA


USA


USA


USA


Japan
USA


In [114]:
info_dict;

In [201]:
info_dict['DQ811784.2'];

### make df from dict

In [346]:
info_df = pd.DataFrame(info_dict).T

In [347]:
info_df['organism'] = 'BCoV'

In [348]:
info_df.rename(columns={"acv": "accession"}, inplace=True)

In [349]:
info_df.columns

Index(['accession', 'comment', 'created', 'definition', 'strain', 'organism',
       'mol_type', 'isolate', 'isolation_source', 'db_xref', 'country',
       'collection_date', 'collected_by', 'gene', 'gi', 'length', 'locus',
       'moltype', 'other_seqids', 'sequence', 'updated', 'date', 'region',
       'host', 'division', 'note', 'serotype'],
      dtype='object')

In [392]:
info_df.serotype.value_counts()

Bovine    1
Name: serotype, dtype: int64

In [383]:
info_df[['locus', 'accession']]

,locus,accession
MW711287.1,MW711287,MW711287.1
MN982199.1,MN982199,MN982199.1
MN982198.1,MN982198,MN982198.1
NC_003045.1,NC_003045,NC_003045.1
LC494178.1,LC494178,LC494178.1
...,...,...
FJ938064.1,FJ938064,FJ938064.1
FJ938063.1,FJ938063,FJ938063.1
U00735.2,BCU00735,U00735.2
AB354579.1,AB354579,AB354579.1


 - definition: redundant as strain information was extracted and organism information are all BCoV and were filled in as such in column 'organism'<br>
 - isolate: redundant as incomplete and when filled in it has the same information as 'strain' (whose information was extracted from 'definition')<br>
 - gene: redundant, as all None or 'No information'<br>
 - locus: redundant as it is the accession number without the version number (e.g locus=DQ915164; accession=DQ915164.2)<br>
 - moltype: redundant - as mol_type, only less information ('RNA' vs. 'genomic RNA')<br>
 - sequence: tmi - saved in txt file not used for metadata

In [351]:
info_df.columns

Index(['accession', 'comment', 'created', 'definition', 'strain', 'organism',
       'mol_type', 'isolate', 'isolation_source', 'db_xref', 'country',
       'collection_date', 'collected_by', 'gene', 'gi', 'length', 'locus',
       'moltype', 'other_seqids', 'sequence', 'updated', 'date', 'region',
       'host', 'division', 'note', 'serotype'],
      dtype='object')

### select columns for NextStrain

In [396]:
df_meta = info_df[['accession', 'comment', 'created', 'strain', 'organism',
       'mol_type', 'isolation_source', 'db_xref', 'country',
       'collection_date', 'collected_by', 'gi', 'length', 
       'other_seqids', 'updated', 'date', 'region',
       'host', 'division', 'note', 'serotype']]

In [398]:
df_meta.to_csv('metadata_unsorted.csv', index=False)

### sort columns by number of NaN values in columns

In [407]:
import numpy as np

In [408]:
df = df_meta[df_meta.replace(['None', 'No information', 'No additional information'],[np.nan,np.nan,np.nan]).isna().sum().sort_values().keys()]

In [409]:
df

,accession,created,strain,organism,mol_type,date,db_xref,updated,other_seqids,gi,...,region,country,collection_date,host,division,comment,isolation_source,note,collected_by,serotype
MW711287.1,MW711287.1,15-MAR-2021,SWUN/NMG-D10/2020,BCoV,genomic RNA,2021-03-15,taxon:11128,15-MAR-2021,"{'gb': ['MW711287.1'], 'gi': ['2000170558']}",2000170558,...,Asia,China,01-Sep-2020,NaN,NaN,Sequencing Technology :: Sanger dideoxy sequen...,calf feces,NaN,Cheng Tang,NaN
MN982199.1,MN982199.1,04-NOV-2020,BCOV-China/SWUN/A10/2018,BCoV,genomic RNA,2020-11-04,taxon:11128,04-NOV-2020,"{'gb': ['MN982199.1'], 'gi': ['1925661203']}",1925661203,...,Asia,China,01-Nov-2018,calf,NaN,Sequencing Technology :: Sanger dideoxy sequen...,feces,NaN,Cheng.Tang,NaN
MN982198.1,MN982198.1,04-NOV-2020,BCOV-China/SWUN/A1/2018,BCoV,genomic RNA,2020-11-04,taxon:11128,04-NOV-2020,"{'gb': ['MN982198.1'], 'gi': ['1925661099']}",1925661099,...,Asia,China,01-Nov-2018,calf,NaN,Sequencing Technology :: Sanger dideoxy sequen...,feces,NaN,Cheng.Tang,NaN
NC_003045.1,NC_003045.1,02-AUG-2001,BCoV-ENT,BCoV,genomic RNA,2020-09-17,taxon:11128,17-SEP-2020,"{'ref': ['NC_003045.1'], 'gi': ['15081544']}",15081544,...,No information,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
LC494178.1,LC494178.1,08-FEB-2020,TCG-19,BCoV,genomic RNA,2020-02-08,taxon:11128,08-FEB-2020,"{'dbj': ['LC494178.1'], 'gi': ['1806649953']}",1806649953,...,Asia,Japan,2016-12,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FJ938064.1,FJ938064.1,11-JUL-2009,E-AH187-TC,BCoV,genomic RNA,2009-07-11,taxon:454963,11-JUL-2009,"{'gnl': ['gcv', 'TCVSP-SAIF-00029'], 'gb': ['F...",251748088,...,North America,USA,01-Jan-2000,bovine,Ohio,None,NaN,NaN,NaN,NaN
FJ938063.1,FJ938063.1,11-JUL-2009,E-DB2-TC,BCoV,genomic RNA,2009-07-11,taxon:422216,11-JUL-2009,"{'gnl': ['gcv', 'TCVSP-SAIF-00007'], 'gb': ['F...",251748075,...,North America,USA,30-Nov-1996,bovine,Ohio,None,NaN,NaN,NaN,NaN
U00735.2,U00735.2,02-SEP-1993,Mebus,BCoV,genomic RNA,2003-04-23,taxon:11128,23-APR-2003,"{'gb': ['U00735.2', 'BCU00735'], 'gi': ['30061...",30061510,...,No information,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
AB354579.1,AB354579.1,11-AUG-2007,Kakegawa,BCoV,genomic RNA,2007-08-11,taxon:11128,11-AUG-2007,"{'dbj': ['AB354579.1'], 'gi': ['155369167']}",155369167,...,Asia,Japan,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [410]:
df_meta.to_csv('metadata.csv', index=False)